<a href="https://colab.research.google.com/github/kskksg4/Cat_hipsterizer/blob/master/Cat_hipsterizer_training_lmks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
import numpy as np

import keras, datetime
from keras.layers import Input, Dense
from keras.models import Model
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau
from keras.applications import mobilenet_v2

In [0]:
img_size = 224

In [0]:
data_00 = np.load('/content/gdrive/My Drive/DeepLearning/Cat_hipsterizer/dataset/lmks_CAT_00.npy', allow_pickle=True)
data_01 = np.load('/content/gdrive/My Drive/DeepLearning/Cat_hipsterizer/dataset/lmks_CAT_01.npy', allow_pickle=True)
data_02 = np.load('/content/gdrive/My Drive/DeepLearning/Cat_hipsterizer/dataset/lmks_CAT_02.npy', allow_pickle=True)
data_03 = np.load('/content/gdrive/My Drive/DeepLearning/Cat_hipsterizer/dataset/lmks_CAT_03.npy', allow_pickle=True)
data_04 = np.load('/content/gdrive/My Drive/DeepLearning/Cat_hipsterizer/dataset/lmks_CAT_04.npy', allow_pickle=True)
data_05 = np.load('/content/gdrive/My Drive/DeepLearning/Cat_hipsterizer/dataset/lmks_CAT_05.npy', allow_pickle=True)
data_06 = np.load('/content/gdrive/My Drive/DeepLearning/Cat_hipsterizer/dataset/lmks_CAT_06.npy', allow_pickle=True)

In [0]:
mode = 'lmks'

if mode is 'bbs': 
  output_size = 4
elif mode is 'lmks':
  output_size = 18

start_time = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')

x_train = np.concatenate((data_00.item().get('imgs'),
                          data_01.item().get('imgs'),
                          data_02.item().get('imgs'),
                          data_03.item().get('imgs'),
                          data_04.item().get('imgs'),
                          data_05.item().get('imgs')), axis=0)
y_train = np.concatenate((data_00.item().get(mode), 
                          data_01.item().get(mode), 
                          data_02.item().get(mode),
                          data_03.item().get(mode),
                          data_04.item().get(mode),
                          data_05.item().get(mode)), axis=0)

x_test = np.array(data_06.item().get('imgs'))
y_test = np.array(data_06.item().get(mode))

x_train = x_train.astype('float32') / 255.
x_train = np.reshape(x_train, (-1, img_size, img_size, 3))
x_test = x_test.astype('float32') / 255.
x_test = np.reshape(x_test, (-1, img_size, img_size, 3))

y_train = np.reshape(y_train, (-1, output_size))
y_test = np.reshape(y_test, (-1, output_size))

In [12]:
inputs = Input(shape=(img_size, img_size, 3))

mobilenetv2_model = mobilenet_v2.MobileNetV2(input_shape=(img_size, img_size, 3),
                                            alpha=1.0,
                                            include_top=False,
                                            weights='imagenet',
                                            input_tensor=inputs,
                                            pooling='max')

net = Dense(128, activation='relu')(mobilenetv2_model.layers[-1].output)
net = Dense(64, activation='relu')(net)
net = Dense(output_size, activation='linear')(net)

model = Model(inputs=inputs, outputs=net)

model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_4[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
____________________________________________________________________________________________

In [14]:
model.compile(optimizer=keras.optimizers.Adam(), loss='mse')

model.fit(x_train, y_train, epochs=50, batch_size=32, shuffle=True,
          validation_data=(x_test, y_test), verbose=1)

Train on 8609 samples, validate on 1388 samples
Epoch 1/50
8609/8609 [==============================] - 216s 25ms/step - loss: 748.8550 - val_loss: 1629.2357
Epoch 2/50
8609/8609 [==============================] - 194s 23ms/step - loss: 152.0659 - val_loss: 971.4949
Epoch 3/50
8609/8609 [==============================] - 195s 23ms/step - loss: 82.1358 - val_loss: 466.8180
Epoch 4/50
8609/8609 [==============================] - 195s 23ms/step - loss: 58.7815 - val_loss: 483.9138
Epoch 5/50
8609/8609 [==============================] - 195s 23ms/step - loss: 111.5054 - val_loss: 766.6255
Epoch 6/50
8609/8609 [==============================] - 195s 23ms/step - loss: 58.6619 - val_loss: 558.2467
Epoch 7/50
8609/8609 [==============================] - 195s 23ms/step - loss: 50.4136 - val_loss: 474.0751
Epoch 8/50
8609/8609 [==============================] - 195s 23ms/step - loss: 46.4425 - val_loss: 313.4503
Epoch 9/50
8609/8609 [==============================] - 195s 23ms/step - loss: 39.75

In [0]:
!mkdir models

In [0]:
model.save('models/lmks_1.h5')